# Algorytmy macierzowe - zadanie nr 4 - Eliminacja Gaussa i Cholesky’ego dla macierzy rzadkich

2. Wierszowa rzadka eliminacja Choleskyego w formacie CSR

Marcin Hawryluk, Norbert Wolniak <br>grupa: piątek 12:50B <hr>

In [1]:
import numpy as np
from time import time
import pandas as pd
import matplotlib.pyplot as plt
import os
from time import time
from read_matrix import read_matrix

## Generowanie macierzy

In [2]:
matrices  = []
for file in os.listdir('matrices'):
    matrices.append(read_matrix('matrices/' + file))

## Eliminacja Cholesky'ego dla macierzy gęstych

// wybrać jedną wersję:

In [3]:
def cholesky_LDLT(matrix):
    A = matrix.copy()
    n = A.shape[0]
    
    for k in range(n):
        dkk = A[k, k]
        if abs(dkk) < 1e-8:
            raise ValueError('singular matrix')

        vk = A[k, k+1:].copy()
        A[k, k+1:] /= dkk
        A[k+1:n, k] /= dkk
      
        for j in range(k + 1, n):
            A[j, j:] -= A[k, j:] * vk[j-k-1]
            
    D = np.diag(A)*np.eye(n)
    
    return np.triu(A).T - D + np.eye(n), D 

In [4]:
def cholesky_LLT(matrix):
    A = matrix.copy()
    n = A.shape[0]
    
    for k in range(n):
        if abs(A[k, k]) < 1e-8:
            raise ValueError('singular matrix')
            
        vk = A[k, k+1:n]
        A[k, k] **= 0.5
        dkk = A[k, k]
        A[k, k+1:n] /= dkk
        A[k+1:n, k] /= dkk
        
        
        for j in range(k+1, n):
            A[j, j:n] -= A[k, j:n]*vk[j-k-1]        
        
    return np.triu(A).T

In [5]:
matrix = np.array([[4, 4, 6], [4, 13, 15], [6, 15, 43]], dtype=float)
matrix

array([[ 4.,  4.,  6.],
       [ 4., 13., 15.],
       [ 6., 15., 43.]])

In [6]:
L = cholesky_LLT(matrix)

print("---L L.T---")

print("L:\n", L)
print("\nL.T:\n", L.T)
print("\nL*L.T:\n", L @ L.T)

print("\nnumpy L:\n")
print("", np.linalg.cholesky(matrix), "\n")

print("Correct!" if np.allclose(L @ L.T, matrix) else "wrong")

---L L.T---
L:
 [[2. 0. 0.]
 [2. 3. 0.]
 [3. 3. 5.]]

L.T:
 [[2. 2. 3.]
 [0. 3. 3.]
 [0. 0. 5.]]

L*L.T:
 [[ 4.  4.  6.]
 [ 4. 13. 15.]
 [ 6. 15. 43.]]

numpy L:

 [[2. 0. 0.]
 [2. 3. 0.]
 [3. 3. 5.]] 

Correct!


In [7]:
L, D = cholesky_LDLT(matrix)

print("---L D L.T---")

print("L:\n", L)
print("\nD:\n", D)
print("\nL.T:\n", L.T)
print("\nL*D*L.T:\n", L @ D @ L.T)
print()
print("Correct!" if np.allclose(L @ D @ L.T, matrix) else "wrong")

---L D L.T---
L:
 [[1.  0.  0. ]
 [1.  1.  0. ]
 [1.5 1.  1. ]]

D:
 [[ 4.  0.  0.]
 [ 0.  9.  0.]
 [ 0.  0. 25.]]

L.T:
 [[1.  1.  1.5]
 [0.  1.  1. ]
 [0.  0.  1. ]]

L*D*L.T:
 [[ 4.  4.  6.]
 [ 4. 13. 15.]
 [ 6. 15. 43.]]

Correct!


## Format CSR

Poniższa funkcja służy do konwersji zadanej macierzy do formatu Compressed Sparse Row. 

In [8]:
def convert_to_csr(matrix):
    m, n = matrix.shape
    ICL = []
    VAL = []
    ROWPTR = []
    counter = 0
    
    for i in range(n): # rows
        ROWPTR.append(counter)
        for j in range(m): # columns
            val_ij = matrix[i, j]
            if abs(val_ij) < 1e-8:
                continue
            ICL.append(j)
            VAL.append(val_ij)
            counter += 1
            
    ROWPTR.append(counter)
    
    return ICL, VAL, ROWPTR

## Eliminacja Cholesky'ego dla macierzy rzadkich

## Porównanie czasów

## Wnioski

* 
* 

M. Hawryluk, N. Wolniak. 2021